#### Setup for Open CV endeavour, using predictions from file "for_anil.npz" (Image & predicted mask)
1. Setup : 
    conda install -c menpo opencv3; 
    pip install imutils (Note - conda install imutils fails)
2. OpenCV version: 3.1.0 used

3.  Main concern - predicted mask for size 64 has very low sum np.sum; may I need predicted mask for entire scan in npz format (Tony/Suman has to create prediction in npz format and share the result, which I'll load and use for OpenCV countour detection and finding centriod.      

In [1]:
import cv2 # import the opencv library
import imutils
print("Your OpenCV version: {}".format(cv2.__version__))

Your OpenCV version: 3.1.0


In [2]:
import pandas as pd
import numpy as np
import h5py
import pandas as pd
import argparse
import SimpleITK as sitk
from PIL import Image
import os, glob 
import os, os.path
import tensorflow as tf
import keras
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
from UNET_utils import *
%matplotlib inline

/Users/aluthra/anaconda2/envs/deeplearning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/aluthra/anaconda2/envs/deeplearning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
data_dir = '../data/luna16/'
npz_prediction_file = 'subset2/for_anil.npz'

In [4]:
npz_dict = np.load(data_dir + npz_prediction_file)
print ("Keys in loaded dictionary : ")
npz_dict.keys()

Keys in loaded dictionary : 


['img1',
 'prediction_mask1',
 'prediction_featuremaps0',
 'img0',
 'prediction_featuremaps1',
 'prediction_mask0']

In [5]:
img1 = npz_dict.f.img1.reshape(64,64, 64)
predicted_mask1 = npz_dict.f.prediction_mask1.reshape(64,64, 64)
featuremaps1 = npz_dict.f.prediction_featuremaps1

img0 = npz_dict.f.img0.reshape(64,64, 64)
predicted_mask0 = npz_dict.f.prediction_mask0.reshape(64,64, 64)
featuremaps0 = npz_dict.f.prediction_featuremaps0

#### Discuss : 
1. Predicted mask sum is very low for input image ..?

In [6]:
rounded_predicted_mask1 = np.round(predicted_mask1)
print ("Predicted mask sum (Class1) : {}".format(np.sum(predicted_mask1)))
print ("Rounded Predicted mask sum (Class1): {}".format(np.sum(rounded_predicted_mask1)))

rounded_predicted_mask0 = np.round(predicted_mask0)
print ("\nPredicted mask sum (Class0): {}".format(np.sum(predicted_mask0)))
print ("Rounded Predicted mask sum (Class0): {}".format(np.sum(rounded_predicted_mask0)))

Predicted mask sum (Class1) : 3.1760988235473633
Rounded Predicted mask sum (Class1): 0.0

Predicted mask sum (Class0): 0.005565871950238943
Rounded Predicted mask sum (Class0): 0.0


In [7]:
def displaySlice(class_type, img, mask, sliceNo):
    plt.figure(figsize=[8,8]);    
    plt.subplot(121)
    plt.title('True Image ({})'.format(class_type))
    plt.imshow(img[:, :, sliceNo], cmap='bone');
    
    plt.subplot(122)
    plt.title('Predicted Mask ({})'.format(class_type))
    plt.imshow(mask[:, :, sliceNo], cmap='bone');
#     plt.axis("off")
    plt.show()

In [8]:
interact(displaySlice,class_type=fixed('Class1'),img=fixed(img1), \
         mask=fixed(predicted_mask1), sliceNo=(0,img1.shape[2]-1,1) );
# interact(displaySlice,class_type=fixed('Class0'),img=fixed(img0), \
#          mask=fixed(predicted_mask0), sliceNo=(0,img0.shape[2]-1,1) );

##### Discussion :
1. CV2: using Rounded_predicted_mask1....? 
2. Process 3D image in 2D slice's and then find countour in each 2D slice...? 
3. Convert image from BGR to RGB as OpenCV represents RGB images in reverse order by: 
    cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
4. Note the np.sum(predicted mask) is very low

In [9]:
def morphImage(img, kernel = np.ones((7,7), np.uint8)):
    img_dilated = cv2.dilate(img, kernel, iterations=1)
    img_eroded = cv2.erode(img_dilated, kernel, iterations=1)
    return (img_dilated, img_eroded)

In [14]:
# def findCentriod(cntrs, z_slice, MIN_THRESH = 7): #nodule area for diameter 3 is Pi*1.5*1.5 (i.e. 7) can be ignored
def findCentriod(cntrs, z_slice, MIN_THRESH = 0): #nodule area for diameter 3 is Pi*1.5*1.5 (i.e. 7) can be ignored    
    for c in cntrs:
        if cv2.contourArea(c) > MIN_THRESH:
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cZ = z_slice
            area = int(M["m00"])
            diameter = 2*(np.sqrt(area/np.pi))
#             writer.writerow(cX, cY, area, diameter)
            print(" Coord X = {}; Coord Y = {}; Coord Z = {}; Nodule Area = {}; Diameter = {} ".\
                  format(cX, cY, cZ, area, diameter))

In [15]:
def plot_morphImage(rslt):
    plt.figure(figsize=[8,8]);    
    plt.subplot(121)
    plt.title("Image after Dilation only");
    plt.imshow(rslt[0]);

    plt.subplot(122)
    plt.title("Image after Dilation & Erosion");
    plt.imshow(rslt[1]);

In [16]:
def process_mask(predicted_mask_tst):
    found_contour = False;
    for idx in range(predicted_mask_tst.shape[2]):
    # for idx in range(22, 40, 1):  
        one_slice = predicted_mask_tst[:, :, idx] 
        rslt = morphImage(one_slice)
        # plot_morphImage(rslt)

        tmp_slice = rslt[1]
        cv2.imwrite("tmp_slice.png", tmp_slice)
        image = cv2.imread("tmp_slice.png")

        # #AL....?
        # convert image from BGR to RGB as OpenCV represents RGB images in reverse order
        # cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        cntrs = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cntrs = cntrs[0] if imutils.is_cv2() else cntrs[1]
        if len(cntrs) > 0:
            found_contour = True;
        findCentriod(cntrs, idx)
        
    print("Found contour in mask : {}". format(found_contour))

##### Section A - testing with Simulated mask on image size (64,64,64) with Maks diameter = 12
1. Printing centriod coordinates X, Y , Z (as slice number)....?  

In [17]:
predicted_mask_Simulated = makeMasks()[6].reshape(64,64,64)
process_mask(predicted_mask_Simulated)

 Coord X = 32; Coord Y = 32; Coord Z = 27; Nodule Area = 28; Diameter = 5.970821321441846 
 Coord X = 32; Coord Y = 32; Coord Z = 28; Nodule Area = 56; Diameter = 8.444016491289505 
 Coord X = 32; Coord Y = 32; Coord Z = 29; Nodule Area = 74; Diameter = 9.706684619910241 
 Coord X = 32; Coord Y = 32; Coord Z = 30; Nodule Area = 84; Diameter = 10.341765891652821 
 Coord X = 32; Coord Y = 32; Coord Z = 31; Nodule Area = 92; Diameter = 10.823032759612019 
 Coord X = 32; Coord Y = 32; Coord Z = 32; Nodule Area = 96; Diameter = 11.055812783082736 
 Coord X = 32; Coord Y = 32; Coord Z = 33; Nodule Area = 92; Diameter = 10.823032759612019 
 Coord X = 32; Coord Y = 32; Coord Z = 34; Nodule Area = 84; Diameter = 10.341765891652821 
 Coord X = 32; Coord Y = 32; Coord Z = 35; Nodule Area = 74; Diameter = 9.706684619910241 
 Coord X = 32; Coord Y = 32; Coord Z = 36; Nodule Area = 56; Diameter = 8.444016491289505 
 Coord X = 32; Coord Y = 32; Coord Z = 37; Nodule Area = 28; Diameter = 5.97082132144

##### Section B - using predicted mask for scan 
1. No contours found though slider show litte blips of 1's around slice#33 

In [16]:
process_mask(rounded_predicted_mask1)

Found contour in mask : False


#### Result/Discussion on Erosion & Dilation operations
1. Perform dilation (to increase white region as we are getting very small sizes)
2. Then performing Erosion operation